In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
import seaborn as sns
import tensorflow as tf
import math

from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras import initializers

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
import xgboost as xgb
import sklearn
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_absolute_error, accuracy_score, roc_auc_score
from sklearn.model_selection import GridSearchCV

In [7]:
class Xgboost_solar:
    
    train_a = None
    train_b = None
    train_c = None
    
    observed_a = None
    observed_b = None
    observed_c = None
    
    estimated_a = None
    estimated_b = None
    estimated_c = None
    
    test_a = None
    test_b = None
    test_c = None

    
    def __init__(self, train_a, train_b, train_c, observed_a, 
                 observed_b, observed_c, estimated_a, estimated_b, estimated_c, test_a, test_b, test_c):
        
        self.train_a = train_a
        self.train_b = train_b
        self.train_c = train_c
        
        self.observed_a = observed_a
        self.observed_b = observed_b
        self.observed_c = observed_c
        
        self.estimated_a = estimated_a.drop(["date_calc"], axis=1)
        self.estimated_b = estimated_b.drop(["date_calc"], axis=1)
        self.estimated_c = estimated_c.drop(["date_calc"], axis=1)

        self.test_a = test_a.drop(["date_calc"], axis=1)
        self.test_a = self.test_a .rename(columns={'date_forecast': 'date'})
        
        self.test_b = test_b.drop(["date_calc"], axis=1)
        self.test_b = self.test_b .rename(columns={'date_forecast': 'date'})

        self.test_c = test_c.drop(["date_calc"], axis=1)
        self.test_c = self.test_c.rename(columns={'date_forecast': 'date'})


    def quartersToHours(self, df):
        df['date_forecast'] = pd.to_datetime(df['date_forecast'], format='%Y-%m.%d %H:%M:%S')
        df["year"] = df['date_forecast'].dt.year
        df["month"] = df['date_forecast'].dt.month
        df["day"] = df['date_forecast'].dt.day
        df["hour"] = df['date_forecast'].dt.hour


        group = df.groupby([df["year"], df["month"], df["day"], df["hour"]])  
        result = group.mean()
        result = result.reset_index()

        return_df = result.drop(['year','month', 'day', 'hour'], axis=1)

        return return_df
    
    
    def prepare_data(self):
        
        self.observed_a = self.quartersToHours(self.observed_a)
        self.observed_b = self.quartersToHours(self.observed_b)
        self.observed_c = self.quartersToHours(self.observed_c)
        
        self.estimated_a = self.quartersToHours(self.estimated_a)
        self.estimated_b = self.quartersToHours(self.estimated_b)
        self.estimated_c = self.quartersToHours(self.estimated_c)

        self.test_a = self.quartersToHours(self.test_a)
        self.test_b = self.quartersToHours(self.test_b)
        self.test_c = self.quartersToHours(self.test_c)

        
        
        




In [5]:
train_a = pd.read_parquet('dataset/A/train_targets.parquet')
train_b = pd.read_parquet('dataset/B/train_targets.parquet')
train_c = pd.read_parquet('dataset/C/train_targets.parquet')

X_train_observed_a = pd.read_parquet('dataset/A/X_train_observed.parquet')
X_train_observed_b = pd.read_parquet('dataset/B/X_train_observed.parquet')
X_train_observed_c = pd.read_parquet('dataset/C/X_train_observed.parquet')

X_train_estimated_a = pd.read_parquet('dataset/A/X_train_estimated.parquet')
X_train_estimated_b = pd.read_parquet('dataset/B/X_train_estimated.parquet')
X_train_estimated_c = pd.read_parquet('dataset/C/X_train_estimated.parquet')

X_test_estimated_a = pd.read_parquet('dataset/A/X_test_estimated.parquet')
X_test_estimated_b = pd.read_parquet('dataset/B/X_test_estimated.parquet')
X_test_estimated_c = pd.read_parquet('dataset/C/X_test_estimated.parquet')



In [8]:

model = Xgboost_solar(train_a, train_b, train_c, X_train_observed_a, X_train_observed_b, X_train_observed_c,
                      X_train_estimated_a, X_train_estimated_b, X_train_estimated_c, X_test_estimated_a, 
                      X_test_estimated_b, X_test_estimated_c)

model.prepare_data()


NameError: name 'data' is not defined